# Final Project

Student 1: Alejandro González Álvarez

NIA 1: 252658

Student 2: Luca Franceschi

NIA 2: 253885

Student 3: Júlia Othats-Dalès

NIA 3: 254435

In [1]:
# ideas

'''
classification of doppelgangers: to which person are you more alike?

input: one facial image
output: closest face in the feature space

pretrained model for image recognition?
triplet loss?
siamese network?

use transfer learning or train with vggface2?
fine tune with smaller dataset of doppelgangers

contrastive learning
https://encord.com/blog/guide-to-contrastive-learning/
https://www.v7labs.com/blog/contrastive-learning-guide
https://arxiv.org/pdf/1512.03385
https://www.geeksforgeeks.org/residual-networks-resnet-deep-learning/
https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py
https://www.v7labs.com/blog/triplet-loss
https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html
https://medium.com/@Skpd/triplet-loss-on-imagenet-dataset-a2b29b8c2952
https://omoindrot.github.io/triplet-loss
https://arxiv.org/pdf/1503.03832v3
'''

'\nclassification of doppelgangers: to which person are you more alike?\n\ninput: one facial image\noutput: closest face in the feature space\n\npretrained model for image recognition?\ntriplet loss?\nsiamese network?\n\nuse transfer learning or train with vggface2?\nfine tune with smaller dataset of doppelgangers\n\ncontrastive learning\nhttps://encord.com/blog/guide-to-contrastive-learning/\nhttps://www.v7labs.com/blog/contrastive-learning-guide\nhttps://arxiv.org/pdf/1512.03385\nhttps://www.geeksforgeeks.org/residual-networks-resnet-deep-learning/\nhttps://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py\nhttps://www.v7labs.com/blog/triplet-loss\n'

In [2]:
import torch
import torch.nn as nn
from torchinfo import summary
from torchvision.models import ResNet18_Weights

In [3]:
# Comment if not in Google Colab
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/My Drive/DeepLearning_2024/P4/Data/'
# results_path = '/content/drive/My Drive/DeepLearning_2024/P4/Results/'

# Comment if in Google Colab
data_path = 'Data/'
results_path = 'Results/'

In [4]:
# Print if gpu acceleration is enabled
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Visualizing the datasets

# Database and dataloaders

## Mean and std of the datasets

# ResNet-18

This is our re-implementation of the ResNet-18 architecture extracted from [the torchvision github repository](https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py). We believe that it is important to note that it is not a simple copy-paste, we have put our best efforts in understanding and re-implementing it to fufill our needs.

## Basic ResNet building block

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1, downsample=None, norm_layer=None):
        super(ResidualBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = norm_layer(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, bias=False) # in this one stride = 1 then image shape is kept
        self.bn2 = norm_layer(planes)
        
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x): # Tip for improving performance merge all into one line relu(bn2(conv2(relu(bn1(conv1(x))))))
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

## ResNet Architecture

In [6]:
class ResNet(nn.Module):

    def __init__(self, layers, num_classes=1000, zero_init_residual=False, norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        # Introduction layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Layers
        self.layer1 = self._make_layer(64, 64, layers[0])
        self.layer2 = self._make_layer(64, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(128, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(256, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Last fully connected layer
        self.fc = nn.Linear(512, num_classes)

        # What is going on ???
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        # if zero_init_residual:
        #     for m in self.modules():
        #         if isinstance(m, Bottleneck):
        #             nn.init.constant_(m.bn3.weight, 0)
        #         elif isinstance(m, BasicBlock):
        #             nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, in_planes, out_planes, blocks, stride=1):
        '''
        Creates a layer containing {blocks} ResidualBlocks, where the first one downsamples the output if needed,
        and the next {blocks-1} stacked blocks keep the amount of filters unaltered.
        '''
        norm_layer = self._norm_layer
        downsample = None
        if stride != 1 or in_planes != out_planes:
            downsample = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                norm_layer(out_planes)
            )

        layers = []
        layers.append(ResidualBlock(in_planes, out_planes, stride, downsample, norm_layer))

        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_planes, out_planes, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [7]:
summary(ResNet([2, 2, 2, 2], 1000), input_size=(128,3,256,256))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [128, 1000]               --
├─Conv2d: 1-1                            [128, 64, 128, 128]       9,408
├─BatchNorm2d: 1-2                       [128, 64, 128, 128]       128
├─ReLU: 1-3                              [128, 64, 128, 128]       --
├─MaxPool2d: 1-4                         [128, 64, 64, 64]         --
├─Sequential: 1-5                        [128, 64, 64, 64]         --
│    └─ResidualBlock: 2-1                [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-1                  [128, 64, 64, 64]         36,864
│    │    └─BatchNorm2d: 3-2             [128, 64, 64, 64]         128
│    │    └─ReLU: 3-3                    [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-4                  [128, 64, 64, 64]         36,864
│    │    └─BatchNorm2d: 3-5             [128, 64, 64, 64]         128
│    │    └─ReLU: 3-6                    [128, 64, 64, 64]         --
│

## How pretrained weights are loaded

In [8]:
pretrained_weights = ResNet18_Weights.DEFAULT.get_state_dict()
net = ResNet([2, 2, 2, 2])
net.load_state_dict(pretrained_weights)
# remove fc layer
# fishernet vgg16


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\alex4/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 15.9MB/s]


<All keys matched successfully>

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

## Training loop

In [17]:
def validate(net, val_loader):
    net.eval()

    loss_list = []

    criterion = nn.TripletMarginLoss()

    val_loss_epoch = 0

    for anchors, positives, negatives in val_loader: # batches of positive, negative and anchor images

        anchors = anchors.to(device)
        positives = positives.to(device)
        negatives = negatives.to(device)

        out_anc = net(anchors)
        out_pos = net(positives)
        out_neg = net(negatives)

        loss = criterion(out_anc, out_pos, out_neg)

        val_loss_epoch += loss.cpu().item()
        
        loss_list.append(loss.cpu().item())

    return val_loss_epoch / len(val_loader), loss_list

In [18]:
def fit(net, tr_loader, val_loader, epochs=10, optimizer=None, model_name='default.ckpt'):
    
    net.train()
    
    loss_list = {
        'training': [],
        'validation': []
    }

    criterion = nn.TripletMarginLoss()

    early_stopper = EarlyStopper(patience=3, min_delta=10)

    for e in range(0, epochs):

        loss_list['training'].append([])
        loss_list['validation'].append([])

        tr_loss_epoch = 0

        for anchors, positives, negatives in tr_loader: # batches of positive, negative and anchor images

            anchors = anchors.to(device)
            positives = positives.to(device)
            negatives = negatives.to(device)

            out_anc = net(anchors)
            out_pos = net(positives)
            out_neg = net(negatives)

            loss = criterion(out_anc, out_pos, out_neg)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tr_loss_epoch += loss.cpu().item()
            
            loss_list['training'][e].append(loss.cpu().item())

        tr_loss_epoch /= len(tr_loader)

        val_loss_epoch, val_loss_list = validate(net, val_loader)
        loss_list['validation'][e].append(val_loss_list)

        print('Epoch [{:4d}/{:4d}] | Train Loss: {:.4f} | Validation Loss: {:4d}'.format(e+1, epochs, tr_loss_epoch, val_loss_epoch))
        
        # save trained model
        torch.save(net.state_dict(), results_path + '/' + model_name)

        if early_stopper.early_stop(val_loss_epoch):
            break